In [134]:
import os

import mlflow
from mlflow import MlflowClient
from mlflow.server import get_app_client

In [2]:

%pwd

'c:\\Edu\\Python\\MLOPS_projects\\mlops_nlp_summarizer\\research'

In [3]:
NUMEXPR_MAX_THREADS = 20

In [4]:
#mlflow.environment_variables.MLFLOW_HTTP_REQUEST_MAX_RETRIES = 20

In [5]:
#mlflow.environment_variables.MLFLOW_HTTP_REQUEST_TIMEOUT= 700

In [6]:
os.chdir("../")

In [7]:
%pwd

'c:\\Edu\\Python\\MLOPS_projects\\mlops_nlp_summarizer'

In [136]:
from dataclasses import dataclass
from pathlib import Path


In [137]:
@dataclass(frozen=True)
class CredentialsConfig:
    MLFLOW_TRACKING_URI: str
    MLFLOW_TRACKING_USERNAME: str
    MLFLOW_TRACKING_PASSWORD: str
    MLFLOW_TRACKING_TOKEN: str
    MLFLOW_HTTP_REQUEST_MAX_RETRIES: str
    MLFLOW_HTTP_REQUEST_TIMEOUT: str



In [138]:
from mlops_NLP_Text_Summarization.constants import *
from mlops_NLP_Text_Summarization.utils.common import read_yaml, create_directories, save_json

In [139]:
class Credentials:
    def __init__(
        self,
        secrets_filepath = SECRETS_FILE_PATH):
        

        self.secret= read_yaml(secrets_filepath)
    
    def get_mlflow_tracking_credentials(self) -> CredentialsConfig:
        secret = self.secret
        
        model_evaluation_config = CredentialsConfig(
            MLFLOW_TRACKING_URI=self.secret.MLFLOW_TRACKING_URI,
            MLFLOW_TRACKING_USERNAME=self.secret.MLFLOW_TRACKING_USERNAME,
            MLFLOW_TRACKING_PASSWORD = self.secret.MLFLOW_TRACKING_PASSWORD,
            MLFLOW_TRACKING_TOKEN = self.secret.DAGSHUB_API_TOKEN,
            MLFLOW_HTTP_REQUEST_MAX_RETRIES=self.secret.MLFLOW_HTTP_REQUEST_MAX_RETRIES,
            MLFLOW_HTTP_REQUEST_TIMEOUT=self.secret.MLFLOW_HTTP_REQUEST_TIMEOUT

           
        )
        return {
            "MLFLOW_TRACKING_URI": model_evaluation_config.MLFLOW_TRACKING_URI,
            "MLFLOW_TRACKING_USERNAME": model_evaluation_config.MLFLOW_TRACKING_USERNAME,
            "MLFLOW_TRACKING_PASSWORD": model_evaluation_config.MLFLOW_TRACKING_PASSWORD,
            "MLFLOW_TRACKING_TOKEN": model_evaluation_config.MLFLOW_TRACKING_TOKEN,
            "MLFLOW_HTTP_REQUEST_MAX_RETRIES": model_evaluation_config.MLFLOW_HTTP_REQUEST_MAX_RETRIES,
            "MLFLOW_HTTP_REQUEST_TIMEOUT": model_evaluation_config.MLFLOW_HTTP_REQUEST_TIMEOUT
        }
    

In [140]:
credentials = Credentials()

[2023-08-19 06:55:13,877: INFO: common: yaml file: secrets.yaml loaded successfully]


In [141]:
mlflow_credentials = credentials.get_mlflow_tracking_credentials()

In [142]:
os.environ["MLFLOW_TRACKING_URI"] = mlflow_credentials["MLFLOW_TRACKING_URI"]

In [143]:
os.environ["MLFLOW_TRACKING_USERNAME"] = mlflow_credentials["MLFLOW_TRACKING_USERNAME"]

In [144]:
os.environ["MLFLOW_TRACKING_PASSWORD"] = mlflow_credentials["MLFLOW_TRACKING_PASSWORD"]

In [145]:
os.environ["MLFLOW_HTTP_REQUEST_MAX_RETRIES"] = mlflow_credentials["MLFLOW_HTTP_REQUEST_MAX_RETRIES"]

In [146]:
os.environ["MLFLOW_HTTP_REQUEST_TIMEOUT"] = mlflow_credentials["MLFLOW_HTTP_REQUEST_TIMEOUT"]

In [147]:
os.environ["MLFLOW_TRACKING_TOKEN"] = mlflow_credentials["MLFLOW_TRACKING_TOKEN"]

In [148]:
PROTOCOL_BUFFERS_PYTHON_IMPLEMENTATION="python"

In [188]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    data_path: Path
    model_path: Path
    params: dict
    tokenizer_path: Path
    metric_file_name: Path
    mlflow_uri: str
    mlflow_username: str
    mlflow_password: str
    dagsub_api_token: str
    experiment_name: str
    model_path_packed: Path
    #headers: dict

In [189]:
from mlops_NLP_Text_Summarization.constants import *
from mlops_NLP_Text_Summarization.utils.common import read_yaml, create_directories, save_json


In [190]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        secrets_filepath = SECRETS_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
        ):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.secrets = read_yaml(secrets_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.TrainingArguments
        secrets = self.secrets

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            data_path=config.data_path,
            model_path = config.model_path,
            tokenizer_path = config.tokenizer_path,
            params = params,            
            metric_file_name = config.metric_file_name,
            mlflow_uri = secrets.MLFLOW_TRACKING_URI,
            mlflow_username = secrets.MLFLOW_TRACKING_USERNAME,
            mlflow_password = secrets.MLFLOW_TRACKING_PASSWORD,
            dagsub_api_token = secrets.DAGSHUB_API_TOKEN,
            experiment_name=config.experiment_name,
            model_path_packed= config.model_path_packed,
            #headers = config.headers,
            
        )

        return model_evaluation_config

In [191]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from datasets import load_dataset, load_from_disk, load_metric
import torch
import pandas as pd
from tqdm import tqdm
from urllib.parse import urlparse
import json
import joblib
import yaml

In [192]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config


    
    def generate_batch_sized_chunks(self,list_of_elements, batch_size):
        """split the dataset into smaller batches that we can process simultaneously
        Yield successive batch-sized chunks from list_of_elements."""
        for i in range(0, len(list_of_elements), batch_size):
            yield list_of_elements[i : i + batch_size]

    
    def calculate_metric_on_test_ds(self,dataset, metric, model, tokenizer, 
                               batch_size=16, device="cuda" if torch.cuda.is_available() else "cpu", 
                               column_text="article", 
                               column_summary="highlights"):
        article_batches = list(self.generate_batch_sized_chunks(dataset[column_text], batch_size))
        target_batches = list(self.generate_batch_sized_chunks(dataset[column_summary], batch_size))

        for article_batch, target_batch in tqdm(
            zip(article_batches, target_batches), total=len(article_batches)):
            
            inputs = tokenizer(article_batch, max_length=1024,  truncation=True, 
                            padding="max_length", return_tensors="pt")
            
            summaries = model.generate(input_ids=inputs["input_ids"].to(device),
                            attention_mask=inputs["attention_mask"].to(device), 
                            length_penalty=0.8, num_beams=8, max_length=128)
            ''' parameter for length penalty ensures that the model does not generate sequences that are too long. '''
            
            #  We decode the generated texts, 
            # replace the  token, and add the decoded texts with the references to the metric.
            decoded_summaries = [tokenizer.decode(s, skip_special_tokens=True, 
                                    clean_up_tokenization_spaces=True) 
                for s in summaries]      
            
            decoded_summaries = [d.replace("", " ") for d in decoded_summaries]
            
            
            metric.add_batch(predictions=decoded_summaries, references=target_batch)
            
        #  Finally compute and return the ROUGE scores.
        score = metric.compute()
        return score


    def evaluate(self):
        device = "cuda" if torch.cuda.is_available() else "cpu"
        tokenizer = AutoTokenizer.from_pretrained(self.config.tokenizer_path)
        model_pegasus = AutoModelForSeq2SeqLM.from_pretrained(self.config.model_path).to(device)
       
        #loading data 
        dataset_samsum_pt = load_from_disk(self.config.data_path)
        

        #if mlflow.active_run():
        #    mlflow.end_run()
            
        mlflow.set_tracking_uri(self.config.mlflow_uri)
        #with mlflow.start_run(run_name=self.config.experiment_name) as run:
        
    
            
        #    mlflow.log_params(self.config.params)
            
        rouge_names = ["rouge1", "rouge2", "rougeL", "rougeLsum"]
    
        rouge_metric = load_metric('rouge')

        score = self.calculate_metric_on_test_ds(
        dataset_samsum_pt['test'][0:10], rouge_metric, model_pegasus, tokenizer, batch_size = 2, column_text = 'dialogue', column_summary= 'summary')
            

            # MLflow >= 2.0
        #    mlflow.doctor() 
        #    mlflow.pytorch.log_model(model_pegasus, 'model')

        rouge_dict = dict((rn, score[rn].mid.fmeasure ) for rn in rouge_names )
        
        save_json(path=Path(self.config.metric_file_name), data=rouge_dict)        
            
        #    mlflow.log_metric('rouge1', rouge_dict['rouge1'])
        #    mlflow.log_metric('rouge2', rouge_dict['rouge2'])
        #    mlflow.log_metric('rougeL', rouge_dict['rougeL'])
        #    mlflow.log_metric('rougeLsum', rouge_dict['rougeLsum'])

        #    run_id = run.info.run_id
        #    experiment_id = run.info.experiment_id
        #    model_name = "samsum_pegasus_model"
        #    subpath = "text_summarization"
            # build the run URI
        #    run_uri = f'runs:/{run_id}/{subpath}'
        #    mlflow.end_run()
        #    print(f'artifact_uri= {mlflow.get_artifact_uri()}')
        #    print(f'run_uri= {run_uri}')






In [195]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.evaluate()
except Exception as e:
    raise e
    

[2023-08-19 08:08:20,687: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-08-19 08:08:20,690: INFO: common: yaml file: params.yaml loaded successfully]
[2023-08-19 08:08:20,692: INFO: common: yaml file: secrets.yaml loaded successfully]
[2023-08-19 08:08:20,693: INFO: common: created directory at: artifacts]
[2023-08-19 08:08:20,694: INFO: common: created directory at: artifacts/model_evaluation]


C:\Users\Admin\AppData\Local\Temp\ipykernel_10756\2333105942.py:69: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  rouge_metric = load_metric('rouge')
  0%|          | 0/5 [00:07<?, ?it/s]


KeyboardInterrupt: 

In [27]:
import requests
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

def http_request_with_retry(url, retries=3, backoff_factor=0.5, status_forcelist=(401, 500, 502, 504)):
    """
    Performs an HTTP request with an automatic retry policy using Python's requests module.
    :param url: The URL to request.
    :param retries: The maximum number of retries to attempt.
    :param backoff_factor: The backoff factor to use between retries.
    :param status_forcelist: A tuple of HTTP status codes that should trigger a retry.
    :return: The response object.
    """
    session = requests.Session()
    retry = Retry(total=retries, backoff_factor=backoff_factor, status_forcelist=status_forcelist)
    adapter = HTTPAdapter(max_retries=retry)
    session.mount('http://', adapter)
    session.mount('https://', adapter)
    response = session.get(url)
    return response

url = 'https://dagshub.com/Nkifor/mlops-nlp-text-summarizer.mlflow/api/2.0/mlflow/runs/create'
response = http_request_with_retry(url)

print(response.status_code)
print(response.json())

RetryError: HTTPSConnectionPool(host='dagshub.com', port=443): Max retries exceeded with url: /Nkifor/mlops-nlp-text-summarizer.mlflow/api/2.0/mlflow/runs/create (Caused by ResponseError('too many 401 error responses'))